In [21]:
import pandas as pd 
import numpy as np
from pathlib import Path
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from statsmodels.regression.rolling import RollingOLS
import math

In [22]:
path = Path("../in-sample")
df_list = []
for file in path.glob("*.csv"):
    df_list.append(pd.read_csv(file))

total = pd.concat(df_list)
total

,index,trade_date,volume,price,volatility,spx_beta,market_cap,avg_market_size,primary_route,industry_group,med_spread_1,med_spread_2,med_spread_3,med_spread_4
0,0,2018-09-06,39482.0,11.965,7.3216,0.017231,20.34,13229.939940,ARCA,NaN,0.010000,0.010000,0.010000,0.010000
1,1,2018-09-06,4706968.0,65.650,23.9744,1.716107,39751.61,759.969758,NASDAQ,NaN,0.037073,0.021642,0.010983,0.010000
2,2,2018-09-06,140047.0,8.030,53.2032,0.481377,197.51,224.762331,NYSE,Healthcare-Services,0.101404,0.041143,0.023961,0.013841
3,3,2018-09-06,41184.0,50.350,15.9040,0.713574,188.81,788.331705,ARCA,NaN,0.278878,0.201292,0.097860,0.072101
4,4,2018-09-06,4152728.0,39.670,29.1968,1.245923,18268.40,618.365490,NASDAQ,Airlines,0.021556,0.013574,0.010000,0.010000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8423,8423,2018-10-08,316778.0,24.900,41.0432,1.350307,635.43,524.290295,NASDAQ,Retail,0.301332,0.096494,0.049769,0.036930
8424,8424,2018-10-08,1598045.0,21.160,63.2768,1.822933,2279.33,418.827669,NYSE,Software,0.110641,0.078839,0.048854,0.025086
8425,8425,2018-10-08,1835585.0,41.820,35.4368,1.670933,8442.02,278.620600,NASDAQ,Internet,0.094725,0.056245,0.019761,0.010000
8426,8426,2018-10-08,100723.5,13.810,58.4576,1.818244,441.37,207.220727,NYSE,Biotechnology,0.219852,0.144525,0.079039,0.047397


#GENERAL STUFF
total["inv_price"] = 1 / total["price"]
total["inv_avg_market_size"] = 1/ total["avg_market_size"]
total["inv_spx_beta"] = 1 / total["spx_beta"]
total["spread-price1"] = total["med_spread_1"] / total["price"]
total["spread-price2"] = total["med_spread_2"] / total["price"]
total["spread-price3"] = total["med_spread_3"] / total["price"]
total["spread-price4"] = total["med_spread_4"] / total["price"]
total["liquidity"] = total["price"] * total["volume"]
total["log_liquidity"] = total["price"] * total["volume"]
total["turnover"] = total["price"] * total["volume"]/total["market_cap"]
total["volatility"] = total["volatility"]
for x in total["log_liquidity"]:
    if x > 1 :
        x = math.log(x)
    

total["pow_liquidity"] = total["liquidity"].apply(lambda x : math.pow(x,0.2))

#get inv_sqrt_liq
#total = total[total["sqrt_liquidity"] > 1]
#total["inv_sqrt_liq"] = 1 / total["sqrt_liquidity"]

print("Percentage of spread-price ratio under 0.2:")
print(100*total[(total["spread-price1"]<0.2)].liquidity.count()/len(total))

#total[(total["spread-price1"]<0.2)].hist(alpha = 0.1)

In [24]:
#different primary routes
things = {}
for i in total["primary_route"]:
    if i in things.keys(): 
        things[i] += 1
    else: 
        things[i] = 1 
things
total

,index,trade_date,volume,price,volatility,spx_beta,market_cap,avg_market_size,primary_route,industry_group,...,inv_avg_market_size,inv_spx_beta,spread-price1,spread-price2,spread-price3,spread-price4,liquidity,log_liquidity,turnover,pow_liquidity
0,0,2018-09-06,39482.0,11.965,7.3216,0.017231,20.34,13229.939940,ARCA,NaN,...,0.000076,58.034089,0.000836,0.000836,0.000836,0.000836,4.724021e+05,4.724021e+05,23225.276794,13.641508
1,1,2018-09-06,4706968.0,65.650,23.9744,1.716107,39751.61,759.969758,NASDAQ,NaN,...,0.001316,0.582714,0.000565,0.000330,0.000167,0.000152,3.090124e+08,3.090124e+08,7773.583239,49.887897
2,2,2018-09-06,140047.0,8.030,53.2032,0.481377,197.51,224.762331,NYSE,Healthcare-Services,...,0.004449,2.077372,0.012628,0.005124,0.002984,0.001724,1.124577e+06,1.124577e+06,5693.774543,16.225492
3,3,2018-09-06,41184.0,50.350,15.9040,0.713574,188.81,788.331705,ARCA,NaN,...,0.001269,1.401396,0.005539,0.003998,0.001944,0.001432,2.073614e+06,2.073614e+06,10982.545416,18.337731
4,4,2018-09-06,4152728.0,39.670,29.1968,1.245923,18268.40,618.365490,NASDAQ,Airlines,...,0.001617,0.802618,0.000543,0.000342,0.000252,0.000252,1.647387e+08,1.647387e+08,9017.687360,43.990524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8423,8423,2018-10-08,316778.0,24.900,41.0432,1.350307,635.43,524.290295,NASDAQ,Retail,...,0.001907,0.740572,0.012102,0.003875,0.001999,0.001483,7.887772e+06,7.887772e+06,12413.282659,23.954707
8424,8424,2018-10-08,1598045.0,21.160,63.2768,1.822933,2279.33,418.827669,NYSE,Software,...,0.002388,0.548567,0.005229,0.003726,0.002309,0.001186,3.381463e+07,3.381463e+07,14835.338542,32.049476
8425,8425,2018-10-08,1835585.0,41.820,35.4368,1.670933,8442.02,278.620600,NASDAQ,Internet,...,0.003589,0.598468,0.002265,0.001345,0.000473,0.000239,7.676416e+07,7.676416e+07,9093.103866,37.759976
8426,8426,2018-10-08,100723.5,13.810,58.4576,1.818244,441.37,207.220727,NYSE,Biotechnology,...,0.004826,0.549981,0.015920,0.010465,0.005723,0.003432,1.390992e+06,1.390992e+06,3151.531674,16.930309


In [25]:
total.groupby('industry_group').trade_date.count().sort_values(ascending=False)

industry_group
Banks                  75215
REITS                  58213
Biotechnology          47516
Pharmaceuticals        42672
Software               30479
                       ...  
Storage/Warehousing      407
Food Service             320
Shipbuilding             147
Equity Fund              147
Multi-National           147
Name: trade_date, Length: 75, dtype: int64

In [26]:
#Different Industries
things = {}
for i in total["industry_group"]:
    if i in things.keys(): 
        things[i] += 1
    else: 
        things[i] = 1 
things

{nan: 394653,
 'Healthcare-Services': 10647,
 'Airlines': 2892,
 'Diversified Finan Serv': 27098,
 'Insurance': 28787,
 'Commercial Services': 27049,
 'Electronics': 16178,
 'Building Materials': 7678,
 'Computers': 15978,
 'Retail': 30459,
 'REITS': 58213,
 'Mining': 16052,
 'Oil&Gas': 28064,
 'Transportation': 20772,
 'Miscellaneous Manufactur': 7983,
 'Agriculture': 3861,
 'Machinery-Constr&Mining': 1725,
 'Pharmaceuticals': 42672,
 'Banks': 75215,
 'Investment Companies': 15859,
 'Biotechnology': 47516,
 'Beverages': 5531,
 'Healthcare-Products': 25418,
 'Household Products/Wares': 2145,
 'Chemicals': 11819,
 'Telecommunications': 23488,
 'Software': 30479,
 'Semiconductors': 13830,
 'Engineering&Construction': 5201,
 'Coal': 2203,
 'Auto Parts&Equipment': 6471,
 'Auto Manufacturers': 2673,
 'Environmental Control': 5505,
 'Electric': 18166,
 'Trucking&Leasing': 1470,
 'Electrical Compo&Equip': 5131,
 'Internet': 25503,
 'Distribution/Wholesale': 4844,
 'Machinery-Diversified': 754

In [27]:
#Removal of Nan
temp = total
total = total.dropna(subset=total.columns.difference(["industry_group"]))

In [37]:
#Create the model and try out stuff 
#DO NOT RUN 

temp = total
print(len(total))


x_col = ["log_liquidity","spx_beta","volatility"]
y_col = ["spread-price1"]

#"liquidity","spx_beta","avg_market_size"

split1 = int(0.35 * len(temp))
split2 = int(0.7 * len(temp))
df_train, df_middle , df_test = temp[:split1], temp[split1:split2] , temp[split2:]

#Outlier Removal Here 
df_train = df_train[((df_train["spread-price1"]<0.2))]

weights = df_train["price"]*df_train["volume"]
weights = df_train["volume"]
weights = weights * df_train["price"]
weights = weights.reset_index(drop=True)
weights = df_train["pow_liquidity"]
#for i in range(len(weights)):
    #weights[i] = 1
#weights = df_train["inv_sqrt_liq"]

#print(f"Training from {df_train.index.date[0]} until {df_train.index.date[-1]}")

x_train = df_train[x_col]
x_middle = df_middle[x_col]
y_train = df_train[y_col]
y_middle = df_middle[y_col]
x_test = df_test[x_col]
y_test = df_test[y_col]

model = LinearRegression(fit_intercept=False)
model.fit(x_train, y_train,sample_weight = weights)

print(model.score(x_train,y_train))
print(model.score(x_middle,y_middle))
print(model.score(x_test,y_test))
df = pd.DataFrame(model.predict(x_test)).rename(columns = {0:"ok"})["ok"].clip(lower=0)
print(model.coef_)
df.to_csv("file.csv")

1044716
0.3272635528107273
-1.7240266549922723e-05
-3.898660896783923e-06
[[-7.42278897e-13 -3.57636991e-03  3.42306445e-04]]


In [29]:
#Columns to train 
x_col = ["volatility","spx_beta","log_liquidity"]
y_col1 = ["spread-price1"]
y_col2 = ["spread-price2"]
y_col3 = ["spread-price3"]
y_col4 = ["spread-price4"]

In [30]:
#Model training 
x_train1 = total[(total["spread-price1"]<0.2)][x_col]
x_train2 = total[(total["spread-price2"]<0.2)][x_col]
x_train3 = total[(total["spread-price3"]<0.2)][x_col]
x_train4 = total[(total["spread-price4"]<0.2)][x_col]
y_train1 = total[(total["spread-price1"]<0.2)][y_col1]
y_train2 = total[(total["spread-price2"]<0.2)][y_col2]
y_train3 = total[(total["spread-price3"]<0.2)][y_col3]
y_train4 = total[(total["spread-price4"]<0.2)][y_col4]
weights1 = total[total["spread-price1"] < 0.2]["pow_liquidity"] 
weights2 = total[total["spread-price2"] < 0.2]["pow_liquidity"] 
weights3 = total[total["spread-price3"] < 0.2]["pow_liquidity"] 
weights4 = total[total["spread-price4"] < 0.2]["pow_liquidity"] 

model1 = LinearRegression(fit_intercept=False)
model1.fit(x_train1, y_train1,sample_weight = weights1)
model2 = LinearRegression(fit_intercept=False)
model2.fit(x_train2, y_train2,sample_weight = weights2)
model3 = LinearRegression(fit_intercept=False)
model3.fit(x_train3, y_train3,sample_weight = weights3)
model4 = LinearRegression(fit_intercept=False)
model4.fit(x_train4, y_train4,sample_weight = weights4)

print(model1.score(x_train1,y_train1))
print(model2.score(x_train2,y_train2))
print(model3.score(x_train3,y_train3))
print(model4.score(x_train4,y_train4))

0.326809737068089
0.3252105498681652
0.3035770298904138
0.26306054613687424


In [32]:
#Model predicting 
path = Path("../out-of-sample")
df_list = []
for file in path.glob("*.csv"):
    if(len(str(file)) == 31):
        df = pd.read_csv(file)
        out_path = "../predictions/" + str(file).split("/")[-1]
        df["log_liquidity"] = df["price"]*df["volume"]
        for x in df["log_liquidity"]:
            if x > 1 :
                x = math.log(x)
        
        df1 = pd.DataFrame(model1.predict(df[x_col])).rename(columns={0:"med_spread_1"})
        df["med_spread_1"] = df1["med_spread_1"]
        df["med_spread_1"] = df["med_spread_1"]*df["price"]
        df["med_spread_1"] = df["med_spread_1"].clip(lower = 0)
        
        df2 = pd.DataFrame(model2.predict(df[x_col])).rename(columns={0:"med_spread_2"})
        df["med_spread_2"] = df2["med_spread_2"]
        df["med_spread_2"] = df["med_spread_2"]*df["price"]
        df["med_spread_2"] = df["med_spread_2"].clip(lower = 0)
        
        df3 = pd.DataFrame(model3.predict(df[x_col])).rename(columns={0:"med_spread_3"})
        df["med_spread_3"] = df3["med_spread_3"]
        df["med_spread_3"] = df["med_spread_3"]*df["price"]
        df["med_spread_3"] = df["med_spread_3"].clip(lower = 0)
        
        df4 = pd.DataFrame(model4.predict(df[x_col])).rename(columns={0:"med_spread_4"})
        df["med_spread_4"] = df4["med_spread_4"]
        df["med_spread_4"] = df["med_spread_4"]*df["price"]
        df["med_spread_4"] = df["med_spread_4"].clip(lower = 0)
        
        df.to_csv(out_path,index = False)
        